# Notebook 01: Document Matching & Source Coverage Audit

Run cells in order.

- Cells 1–4: Create tables (idempotent)
- Cell 4b: Create silver_document_person_override and add manual overrides
- Cell 5: Main matching algorithm
- Cell 6: Diagnostic query for unmatched files — review and resolve before moving on
- Cell 7: Source/doc_type mapping view
- Cell 8: Populate gold_source_coverage
- Cell 9: Coverage summary
- Cell 10: People with most uncovered citations
- Cell 11: Verify a specific person

## Cell 1 — Create silver_person_alias

Derives two alias types:
- MARRIED_NAME: wife takes husband's surname
- FATHER_SURNAME: child surname differs from biological father's surname

In [0]:
-- %sql
CREATE VIEW IF NOT EXISTS genealogy.silver_person_alias AS
SELECT
DISTINCT
  sf.wife_gedcom_id AS person_gedcom_id,
  wp.given_name AS alias_given_name,
  hp.surname AS alias_surname,
  'MARRIED_NAME' AS alias_type,
  CONCAT('Married name via ', coalesce(hp.given_name, '[unknown]'), ' ', hp.surname) AS notes
FROM
  genealogy.silver_family sf
    JOIN genealogy.gold_person_life wp
      ON sf.wife_gedcom_id = wp.person_gedcom_id
    JOIN genealogy.gold_person_life hp
      ON sf.husband_gedcom_id = hp.person_gedcom_id
WHERE
  hp.surname IS NOT NULL
  AND sf.relationship_type is null -- i.e. not 'partner'
  AND hp.surname != wp.surname -- exclude cases where surnames already match
  UNION ALL
  -- Populate father surname aliases for children with different surnames
  SELECT DISTINCT
  sfc.child_gedcom_id                                        AS person_gedcom_id,
  cp.given_name                                              AS alias_given_name,
  fp.surname                                                 AS alias_surname,
  'FATHER_SURNAME'                                           AS alias_type,
  CONCAT('Father surname via ', fp.given_name, ' ', fp.surname) AS notes
FROM genealogy.silver_family_child sfc
JOIN genealogy.silver_family sf    ON sfc.family_gedcom_id  = sf.family_gedcom_id
JOIN genealogy.gold_person_life cp ON sfc.child_gedcom_id   = cp.person_gedcom_id
JOIN genealogy.gold_person_life fp ON sf.husband_gedcom_id  = fp.person_gedcom_id
WHERE cp.surname != fp.surname
  AND fp.surname IS NOT NULL
  AND cp.surname IS NOT NULL
  AND sfc.father_relationship_type = 'biological';

COMMENT ON VIEW genealogy.silver_person_alias IS 'Derived name aliases for matching — married surnames and father surnames.';

## Cell 2 — Create silver_document_person

In [0]:
-- %sql
CREATE TABLE IF NOT EXISTS genealogy.silver_document_person (
  file_id             STRING   NOT NULL,
  file_name           STRING,
  person_gedcom_id    STRING   NOT NULL,
  display_name        STRING,
  match_method        STRING,
  match_confidence    STRING,
  candidate_count     INT,
  notes               STRING,
  matched_at          TIMESTAMP
)
USING DELTA
COMMENT 'Links OCR transcript files to person_gedcom_id.';

## Cell 3 — Create gold_transcript_facts

In [0]:
-- %sql
CREATE TABLE IF NOT EXISTS genealogy.gold_transcript_facts (
  file_id             STRING   NOT NULL,
  person_gedcom_id    STRING,
  fact_type           STRING,
  fact_value          STRING,
  fact_year           INT,
  confidence          STRING,
  source_doc_type     STRING,
  extracted_at        TIMESTAMP
)
USING DELTA
COMMENT 'Structured facts extracted from OCR transcripts by Gemini.';

## Cell 4 — Create gold_fact_comparison and gold_source_coverage

In [0]:
-- %sql
CREATE TABLE IF NOT EXISTS genealogy.gold_fact_comparison (
  person_gedcom_id    STRING,
  display_name        STRING,
  fact_type           STRING,
  tree_value          STRING,
  transcript_value    STRING,
  file_id             STRING,
  file_name           STRING,
  source_doc_type     STRING,
  status              STRING,
  conflict_severity   STRING,
  notes               STRING,
  refreshed_at        TIMESTAMP
)
USING DELTA
COMMENT 'Comparison of facts from OCR transcripts vs gold_person_life tree data.';

CREATE TABLE IF NOT EXISTS genealogy.gold_source_coverage (
  event_id            STRING,
  person_gedcom_id    STRING,
  source_title        STRING,
  source_category     STRING,
  citation_date       STRING,
  citation_text       STRING,
  citation_url        STRING,
  has_document        BOOLEAN,
  has_transcript      BOOLEAN,
  matched_file_ids    STRING,
  coverage_status     STRING,
  refreshed_at        TIMESTAMP
)
USING DELTA
COMMENT 'Source coverage audit: which tree citations have downloaded documents/transcripts.';

## Cell 4b — Create silver_document_person_override

Add manual overrides here for cases the algorithm cannot resolve (e.g. Sr/Jr same-name father/son with overlapping timelines).
Overrides are seeded into silver_document_person before the algorithm runs, and those file_ids are then skipped automatically.

In [0]:
-- %sql
CREATE TABLE IF NOT EXISTS genealogy.silver_document_person_override (
  file_id           STRING NOT NULL,
  file_name         STRING,
  person_gedcom_id  STRING NOT NULL,
  notes             STRING
)
USING DELTA
COMMENT 'Manual overrides for document-to-person matching. Takes precedence over algorithmic matching.';

-- Add overrides as needed:
INSERT INTO genealogy.silver_document_person_override
  VALUES
    (
      '11k8M0wHr4IAl6hW20HBMoWW5FUfK5Tp4',
      'CUTHBERTSON_David_1921_Military.pdf',
      '@I_967018283@',
      'Sr/Jr ambiguity — confirmed via document content'
    ),
    (
      '0B00VuRbH2UFNR24taUhibWhaOU0',
      'CUTHBERTSON_David (Sr)_1911_Census.jpg',
      '@I_967018273@',
      'Sr/Jr ambiguity — confirmed via document content'
    ),
    (
      '0B00VuRbH2UFNNS05X2d6R1Q4Mnc',
      'CUTHBERTSON_David_1926_Marriage.jpg',
      '@I_967018283@',
      'Sr/Jr ambiguity — confirmed via document content'
    ),
    (
      '1wtXVBQJxhhl5RF8fwD92GJXOGml2JfZL',
      'CUTHBERTSON_George (Sr)_1841_Census.jpg',
      '@I998518491@',
      'Sr/Jr ambiguity — confirmed via document content'
    );

SELECT * FROM genealogy.silver_document_person_override;

## Cell 5 — Main matching algorithm

TRUNCATEs silver_document_person and repopulates from scratch each run.
Seeds manual overrides first, then runs algorithmic matching for remaining files.

Ranking priority:
1. Ancestors before non-ancestors
2. Doc type affinity (birth doc near birth year, death doc near death year, marriage doc near marriage event)
3. Ancestral proximity (0 = direct ancestor)
4. Year plausibility
5. person_gedcom_id ASC (deterministic tiebreaker)

In [0]:
-- %sql
TRUNCATE TABLE genealogy.silver_document_person;

-- Seed manual overrides first
INSERT INTO genealogy.silver_document_person
  (file_id, file_name, person_gedcom_id, display_name,
   match_method, match_confidence, candidate_count, notes, matched_at)
SELECT
  o.file_id,
  o.file_name,
  o.person_gedcom_id,
  p.given_name || ' ' || p.surname AS display_name,
  'MANUAL'                         AS match_method,
  'HIGH'                           AS match_confidence,
  1                                AS candidate_count,
  o.notes,
  CURRENT_TIMESTAMP()
FROM genealogy.silver_document_person_override o
JOIN genealogy.gold_person_life p ON o.person_gedcom_id = p.person_gedcom_id;

-- Algorithmic matching
WITH

cleaned_transcripts AS (
  SELECT
    file_id,
    file_name,
    year,
    surname,
    TRIM(REGEXP_REPLACE(forename, '\\s*\\(?(Jr|Sr|Jnr|Snr)\\.?\\)?\\s*$', '')) AS forename_clean,
    doc_type_detected
  FROM genealogy.ocr_transcriptions
),

marriage_years AS (
  SELECT person_gedcom_id, event_year_parsed AS marriage_year
  FROM genealogy.gold_person_event_timeline
  WHERE event_type = 'MARR' AND event_year_parsed IS NOT NULL
),

candidate_counts AS (
  SELECT t.file_id, COUNT(DISTINCT p.person_gedcom_id) AS candidate_count
  FROM cleaned_transcripts t
  JOIN genealogy.gold_person_life p
    ON UPPER(TRIM(t.surname)) = UPPER(TRIM(p.surname))
   AND (
     UPPER(TRIM(t.forename_clean)) = UPPER(TRIM(p.given_name))
     OR UPPER(TRIM(t.forename_clean)) = UPPER(SPLIT(TRIM(p.given_name), ' ')[0])
     OR UPPER(SPLIT(TRIM(t.forename_clean), ' ')[0]) = UPPER(TRIM(p.given_name))
   )
  GROUP BY t.file_id
  UNION
  SELECT t.file_id, COUNT(DISTINCT a.person_gedcom_id) AS candidate_count
  FROM cleaned_transcripts t
  JOIN genealogy.silver_person_alias a
    ON UPPER(TRIM(t.surname)) = UPPER(TRIM(a.alias_surname))
   AND (
     UPPER(TRIM(t.forename_clean)) = UPPER(TRIM(a.alias_given_name))
     OR UPPER(TRIM(t.forename_clean)) = UPPER(SPLIT(TRIM(a.alias_given_name), ' ')[0])
     OR UPPER(SPLIT(TRIM(t.forename_clean), ' ')[0]) = UPPER(TRIM(a.alias_given_name))
   )
  GROUP BY t.file_id
),

candidate_counts_agg AS (
  SELECT file_id, SUM(candidate_count) AS candidate_count
  FROM candidate_counts
  GROUP BY file_id
),

primary_matches AS (
  SELECT
    t.file_id, t.file_name, t.doc_type_detected, t.year,
    p.person_gedcom_id,
    TRIM(p.given_name) || ' ' || TRIM(p.surname) AS display_name,
    p.birth_date, p.death_date,
    FALSE AS matched_via_alias
  FROM cleaned_transcripts t
  JOIN genealogy.gold_person_life p
    ON UPPER(TRIM(t.surname)) = UPPER(TRIM(p.surname))
   AND (
     UPPER(TRIM(t.forename_clean)) = UPPER(TRIM(p.given_name))
     OR UPPER(TRIM(t.forename_clean)) = UPPER(SPLIT(TRIM(p.given_name), ' ')[0])
     OR UPPER(SPLIT(TRIM(t.forename_clean), ' ')[0]) = UPPER(TRIM(p.given_name))
   )
),

alias_matches AS (
  SELECT
    t.file_id, t.file_name, t.doc_type_detected, t.year,
    a.person_gedcom_id,
    pl.given_name || ' ' || pl.surname || ' (matched via ' || a.alias_type || ': ' || a.alias_surname || ')' AS display_name,
    pl.birth_date, pl.death_date,
    TRUE AS matched_via_alias
  FROM cleaned_transcripts t
  JOIN genealogy.silver_person_alias a
    ON UPPER(TRIM(t.surname)) = UPPER(TRIM(a.alias_surname))
   AND (
     UPPER(TRIM(t.forename_clean)) = UPPER(TRIM(a.alias_given_name))
     OR UPPER(TRIM(t.forename_clean)) = UPPER(SPLIT(TRIM(a.alias_given_name), ' ')[0])
     OR UPPER(SPLIT(TRIM(t.forename_clean), ' ')[0]) = UPPER(TRIM(a.alias_given_name))
   )
  JOIN genealogy.gold_person_life pl ON a.person_gedcom_id = pl.person_gedcom_id
),

all_matches AS (
  SELECT * FROM primary_matches
  UNION ALL
  SELECT * FROM alias_matches
),

scored_matches AS (
  SELECT
    m.file_id, m.file_name, m.person_gedcom_id, m.display_name, m.matched_via_alias,
    cc.candidate_count,
    COALESCE(ap.ancestral_proximity, 9999) AS ancestral_proximity,
    ap.person_id IS NOT NULL               AS is_ancestor,
    (m.birth_date IS NULL OR TRY_CAST(m.year AS INT) >= YEAR(m.birth_date) - 2)
      AND (m.death_date IS NULL OR TRY_CAST(m.year AS INT) <= YEAR(m.death_date) + 2)
                                           AS year_plausible,
    CASE
      WHEN m.doc_type_detected IN ('BirthCertificate','BaptismRegister')
        AND m.birth_date IS NOT NULL
        AND ABS(TRY_CAST(m.year AS INT) - YEAR(m.birth_date)) <= 2  THEN 0
      WHEN m.doc_type_detected IN ('BirthCertificate','BaptismRegister')
        AND m.birth_date IS NOT NULL
        AND ABS(TRY_CAST(m.year AS INT) - YEAR(m.birth_date)) <= 5  THEN 1
      WHEN m.doc_type_detected IN ('DeathCertificate','Probate')
        AND m.death_date IS NOT NULL
        AND ABS(TRY_CAST(m.year AS INT) - YEAR(m.death_date)) <= 2  THEN 0
      WHEN m.doc_type_detected IN ('DeathCertificate','Probate')
        AND m.death_date IS NOT NULL
        AND ABS(TRY_CAST(m.year AS INT) - YEAR(m.death_date)) <= 5  THEN 1
      WHEN m.doc_type_detected = 'MarriageCertificate'
        AND EXISTS (
          SELECT 1 FROM marriage_years my
          WHERE my.person_gedcom_id = m.person_gedcom_id
            AND ABS(TRY_CAST(m.year AS INT) - my.marriage_year) <= 2
        )                                                            THEN 0
      ELSE 2
    END AS doc_type_affinity
  FROM all_matches m
  JOIN candidate_counts_agg cc ON m.file_id = cc.file_id
  LEFT JOIN genealogy.gold_ancestral_proximity ap ON m.person_gedcom_id = ap.person_id
  WHERE NOT EXISTS (
    SELECT 1 FROM genealogy.silver_document_person sdp WHERE sdp.file_id = m.file_id
  )
),

ranked AS (
  SELECT
    *,
    ROW_NUMBER() OVER (
      PARTITION BY file_id
      ORDER BY
        is_ancestor DESC,
        doc_type_affinity ASC,
        ancestral_proximity ASC,
        year_plausible DESC,
        person_gedcom_id ASC
    ) AS rank
  FROM scored_matches
  WHERE
    year_plausible = TRUE
    OR NOT EXISTS (
      SELECT 1 FROM scored_matches sm2
      WHERE sm2.file_id = scored_matches.file_id AND sm2.year_plausible = TRUE
    )
)

INSERT INTO genealogy.silver_document_person
  (file_id, file_name, person_gedcom_id, display_name,
   match_method, match_confidence, candidate_count, notes, matched_at)
SELECT
  file_id, file_name, person_gedcom_id, display_name,
  CASE
    WHEN candidate_count = 1 AND NOT matched_via_alias THEN 'EXACT'
    WHEN candidate_count = 1 AND matched_via_alias     THEN 'EXACT_ALIAS'
    WHEN matched_via_alias                             THEN 'EXACT_MULTI_ALIAS'
    ELSE                                                    'EXACT_MULTI'
  END AS match_method,
  CASE
    WHEN candidate_count = 1 AND is_ancestor           THEN 'HIGH'
    WHEN candidate_count = 1                           THEN 'MEDIUM'
    WHEN is_ancestor AND ancestral_proximity = 0
      AND doc_type_affinity = 0                        THEN 'HIGH'
    WHEN is_ancestor AND rank = 1                      THEN 'MEDIUM'
    ELSE                                                    'LOW'
  END AS match_confidence,
  candidate_count,
  CONCAT_WS(' | ',
    CASE WHEN candidate_count > 1 THEN CONCAT(candidate_count, ' candidates') ELSE NULL END,
    CASE WHEN is_ancestor THEN CONCAT('ancestor proximity=', ancestral_proximity) ELSE 'non-ancestor' END,
    CASE WHEN doc_type_affinity < 2 THEN CONCAT('doc_type_affinity=', doc_type_affinity) ELSE NULL END,
    CASE WHEN matched_via_alias THEN 'matched via alias' ELSE NULL END
  ) AS notes,
  CURRENT_TIMESTAMP()
FROM ranked
WHERE rank = 1;

In [0]:
-- %sql
-- Summary of match results
SELECT match_confidence, match_method, COUNT(*) AS n
FROM genealogy.silver_document_person
GROUP BY match_confidence, match_method
ORDER BY n DESC;

## Cell 6 — Diagnostic: why did files not match?

Review this output after cell 5. Resolve unmatched files by:
- Adding to silver_person_alias (new alias type needed)
- Adding to silver_document_person_override (manual hardcode)
- Correcting the filename in Google Drive and updating ocr_transcriptions

Do NOT use fuzzy/Soundex matching — risk of false positives outweighs benefit.

In [0]:
-- %sql
WITH cleaned_transcripts AS (
  SELECT
    file_id, file_name, year, surname,
    TRIM(REGEXP_REPLACE(forename, '\\s*\\(?(Jr|Sr|Jnr|Snr)\\.?\\)?\\s*$', '')) AS forename_clean,
    doc_type_detected
  FROM genealogy.ocr_transcriptions
),
unmatched AS (
  SELECT t.* FROM cleaned_transcripts t
  WHERE NOT EXISTS (
    SELECT 1 FROM genealogy.silver_document_person sdp WHERE sdp.file_id = t.file_id
  )
),
surname_check AS (
  SELECT
    u.file_id, u.file_name, u.surname, u.forename_clean,
    COUNT(DISTINCT p.person_gedcom_id) AS surname_matches
  FROM unmatched u
  LEFT JOIN genealogy.gold_person_life p
    ON UPPER(TRIM(u.surname)) = UPPER(TRIM(p.surname))
  GROUP BY u.file_id, u.file_name, u.surname, u.forename_clean
),
name_check AS (
  SELECT u.file_id, COUNT(DISTINCT p.person_gedcom_id) AS name_matches
  FROM unmatched u
  LEFT JOIN genealogy.gold_person_life p
    ON UPPER(TRIM(u.surname)) = UPPER(TRIM(p.surname))
   AND (
     UPPER(TRIM(u.forename_clean)) = UPPER(TRIM(p.given_name))
     OR UPPER(TRIM(u.forename_clean)) = UPPER(SPLIT(TRIM(p.given_name), ' ')[0])
     OR UPPER(SPLIT(TRIM(u.forename_clean), ' ')[0]) = UPPER(TRIM(p.given_name))
   )
  GROUP BY u.file_id
),
alias_check AS (
  SELECT u.file_id, COUNT(DISTINCT a.person_gedcom_id) AS alias_matches
  FROM unmatched u
  LEFT JOIN genealogy.silver_person_alias a
    ON UPPER(TRIM(u.surname)) = UPPER(TRIM(a.alias_surname))
   AND (
     UPPER(TRIM(u.forename_clean)) = UPPER(TRIM(a.alias_given_name))
     OR UPPER(TRIM(u.forename_clean)) = UPPER(SPLIT(TRIM(a.alias_given_name), ' ')[0])
     OR UPPER(SPLIT(TRIM(u.forename_clean), ' ')[0]) = UPPER(TRIM(a.alias_given_name))
   )
  GROUP BY u.file_id
)
SELECT
  s.file_name,
  s.surname,
  s.forename_clean,
  s.surname_matches,
  n.name_matches,
  a.alias_matches,
  CASE
    WHEN s.surname_matches = 0                      THEN 'SURNAME_NOT_IN_TREE'
    WHEN n.name_matches = 0 AND a.alias_matches = 0 THEN 'FORENAME_NOT_MATCHED'
    WHEN n.name_matches > 0                         THEN 'YEAR_FILTER_ELIMINATED'
    WHEN a.alias_matches > 0                        THEN 'YEAR_FILTER_ELIMINATED_ALIAS'
    ELSE                                                 'UNKNOWN'
  END AS failure_reason
FROM surname_check s
JOIN name_check n ON s.file_id = n.file_id
JOIN alias_check a ON s.file_id = a.file_id
ORDER BY failure_reason, s.surname, s.forename_clean;

## Cell 7 — Source/doc_type mapping view

Maps silver_source title patterns to doc_type_detected vocabulary.
Extend the values list as new source_title values appear when processing additional surname branches.

In [0]:
-- %sql
CREATE OR REPLACE VIEW genealogy.v_source_doctype_map AS
SELECT source_title, doc_type_category FROM (VALUES
  -- England & Wales certificates
  ('Certified Copy of an Entry at Birth',              'BirthCertificate'),
  ('Certified Copy of an Entry of Marriage',           'MarriageCertificate'),
  ('Certified Copy of an Entry of Death',              'DeathCertificate'),
  -- England census
  ('1841 England Census',                              'Census'),
  ('1851 England Census',                              'Census'),
  ('1861 England Census',                              'Census'),
  ('1871 England Census',                              'Census'),
  ('1881 England Census',                              'Census'),
  ('1891 England Census',                              'Census'),
  ('1901 England Census',                              'Census'),
  ('1911 England Census',                              'Census'),
  ('1921 England Census',                              'Census'),
  ('1939 England and Wales Register',                  'Census'),
  -- Scotland census
  ('1841 Scotland Census',                             'Census'),
  ('1851 Scotland Census',                             'Census'),
  ('1861 Scotland Census',                             'Census'),
  ('1871 Scotland Census',                             'Census'),
  ('1881 Scotland Census',                             'Census'),
  ('1891 Scotland Census',                             'Census'),
  ('1901 Scotland Census',                             'Census'),
  ('1921 Scotland Census',                             'Census'),
  -- US census
  ('1930 United States Federal Census',                'Census'),
  ('1940 United States Federal Census',                'Census'),
  ('1950 United States Federal Census',                'Census'),
  -- Baptism / christening registers
  ('England, Select Births and Christenings, 1538-1975',                          'BaptismRegister'),
  ('Scotland, Select Births and Baptisms, 1564-1950',                             'BaptismRegister'),
  -- Marriage registers & OPR
  ('Nottinghamshire, England, Church of England Marriages and Banns, 1754-1937',  'MarriageCertificate'),
  ('Scotland OPR Banns & Marriages, 1553-1854',                                   'MarriageCertificate'),
  -- Death registers
  ('Scotland Statutory Death Register',                                            'DeathCertificate'),
  -- BMD indexes
  ('England & Wales, Birth Index: 1984-2005',                                     'BMDIndex'),
  ('England & Wales, Death Index: 1984-2005',                                     'BMDIndex'),
  ('England & Wales, Marriage Index: 1916-2005',                                  'BMDIndex'),
  ('England & Wales, Civil Registration Marriage Index, 1916-2005',               'BMDIndex'),
  ('England & Wales, FreeBMD Birth Index, 1837-1915',                             'BMDIndex'),
  ('England & Wales, FreeBMD Death Index: 1837-1915',                             'BMDIndex'),
  ('Scotland Statutory Register Births Index, 1855-',                             'BMDIndex'),
  ('Scotland Statutory Register Marriages Index, 1855-1930',                      'BMDIndex'),
  -- Newspapers
  ('UK and Ireland, Newspapers.com™ Marriage Index, 1800s-current',          'NewspaperClipping'),
  -- No file expected
  ('UK, Electoral Registers, 2003-2010',                                          'OTHER_NO_FILE')
) AS t(source_title, doc_type_category);

## Cell 8 — Populate gold_source_coverage

Run after silver_document_person is fully populated and v_source_doctype_map is created.

In [0]:
-- %sql
TRUNCATE TABLE genealogy.gold_source_coverage;

INSERT INTO genealogy.gold_source_coverage (
  event_id, person_gedcom_id, source_title, source_category,
  citation_date, citation_text, citation_url,
  has_document, has_transcript, matched_file_ids, coverage_status, refreshed_at
)
WITH

event_person AS (
  SELECT DISTINCT ep.event_id, ep.person_id AS person_gedcom_id
  FROM genealogy.silver_event_participant ep
),

citations_raw AS (
  -- One row per (event_id, person, source) — used for the final SELECT
  -- to preserve event_id linkage
  SELECT
    es.event_id,
    ep.person_gedcom_id,
    ss.source_title,
    COALESCE(dm.doc_type_category, 'OTHER') AS source_category,
    TRY_CAST(REGEXP_EXTRACT(ss.source_title, '(\\d{4})') AS INT) AS citation_year,
    es.citation_date, es.citation_text, es.citation_url
  FROM genealogy.silver_event_source es
  JOIN event_person ep ON es.event_id = ep.event_id
  LEFT JOIN genealogy.silver_source ss ON es.source_xref = ss.record_xref
  LEFT JOIN genealogy.v_source_doctype_map dm ON ss.source_title = dm.source_title
),

citations_deduped AS (
  -- Collapse to one row per (person, source_title) for coverage matching.
  -- Multiple events citing the same source should produce one coverage result,
  -- not one row per event.
  SELECT DISTINCT
    person_gedcom_id,
    source_title,
    source_category,
    citation_year
  FROM citations_raw
),

doc_matches AS (
  SELECT
    sdp.person_gedcom_id,
    ot.doc_type_detected,
    YEAR(TRY_CAST(ot.year AS DATE))  AS doc_year,
    sdp.file_id,
    ot.transcribed_text IS NOT NULL    AS has_transcript
  FROM genealogy.silver_document_person sdp
  JOIN genealogy.ocr_transcriptions ot ON sdp.file_id = ot.file_id
),

doc_coverage AS (
  SELECT
    c.person_gedcom_id,
    c.source_category,
    c.citation_year,
    COUNT(DISTINCT dm.file_id) > 0                                     AS has_document,
    COUNT(DISTINCT CASE WHEN dm.has_transcript THEN dm.file_id END) > 0 AS has_transcript,
    CONCAT_WS(',', COLLECT_LIST(DISTINCT dm.file_id))                  AS matched_file_ids
  FROM citations_deduped c
  JOIN doc_matches dm
    ON  c.person_gedcom_id = dm.person_gedcom_id
    AND c.source_category  = dm.doc_type_detected
    AND (
      c.source_category != 'Census'
      OR dm.doc_year = c.citation_year
    )
  GROUP BY c.person_gedcom_id, c.source_category, c.citation_year
)

SELECT
  c.event_id, c.person_gedcom_id, c.source_title, c.source_category,
  c.citation_date, c.citation_text, c.citation_url,
  COALESCE(dc.has_document, FALSE)   AS has_document,
  COALESCE(dc.has_transcript, FALSE) AS has_transcript,
  dc.matched_file_ids,
  CASE
    WHEN c.source_category = 'BMDIndex'      THEN 'INDEX_ONLY'
    WHEN c.source_category = 'OTHER_NO_FILE' THEN 'NOT_APPLICABLE'
    WHEN dc.has_transcript = TRUE            THEN 'COVERED'
    WHEN dc.has_document   = TRUE            THEN 'DOCUMENT_NO_TRANSCRIPT'
    ELSE                                          'UNCOVERED'
  END AS coverage_status,
  CURRENT_TIMESTAMP()
FROM citations_raw c
LEFT JOIN doc_coverage dc
  ON  c.person_gedcom_id = dc.person_gedcom_id
  AND c.source_category  = dc.source_category
  AND (
    c.source_category != 'Census'
    OR c.citation_year  = dc.citation_year
  );

## Cell 9 — Coverage summary dashboard

In [0]:
-- %sql
SELECT coverage_status, COUNT(*) AS n
FROM genealogy.gold_source_coverage
GROUP BY coverage_status
ORDER BY n DESC;

## Cell 10 — People with most uncovered citations

In [0]:
-- %sql
SELECT
  p.given_name || ' ' || p.surname as display_name,
  year(p.birth_date) as birth_year,
  year(p.death_date) as death_year,
  b.branch,
  COUNT(*) AS uncovered_citations,
  COLLECT_LIST(DISTINCT sc.source_title) AS missing_source_types
FROM genealogy.gold_source_coverage sc
JOIN genealogy.gold_person_life p ON sc.person_gedcom_id = p.person_gedcom_id
JOIN genealogy.gold_person_branch b ON sc.person_gedcom_id = b.person_gedcom_id
WHERE sc.coverage_status = 'UNCOVERED'
GROUP BY 1,2,3,4
ORDER BY uncovered_citations DESC
LIMIT 50;

## Cell 11 — Verify a specific person

Substitute a real person_gedcom_id from your tree.

In [0]:
-- %sql
SELECT
  sc.source_title,
  sc.source_category,
  sc.citation_date,
  sc.coverage_status,
  sc.matched_file_ids
FROM genealogy.gold_source_coverage sc
WHERE sc.person_gedcom_id = '@I123@'
ORDER BY sc.source_category, sc.citation_date;